In [1]:
import torch
from torch import nn
import pandas as pd
from utils import *
from dataset import CustomEmbeddingDataset
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

In [153]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
device = 'cuda'
# model_id = "sentence-transformers/all-MiniLM-L6-v2"
# model_id = 'sentence-transformers/distilbert-base-nli-mean-tokens'
model_id = 'sentence-transformers/roberta-base-nli-mean-tokens'
model = SentenceTransformer(model_id).to(device)

Downloading:   0%|          | 0.00/683 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/329 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [2]:
train_data = pd.read_csv('data/preprocessed/nli_coarse_train.csv')
train_data['label_count'] = train_data.apply(lambda x: [int(y) for y in string_of_list_to_list(x['label_count'])], axis=1)
val_data = pd.read_csv('data/preprocessed/nli_coarse_val.csv')

In [3]:
train_data.gold_label.value_counts()

entailment       18279
contradiction    17959
neutral          17002
-                 1033
Name: gold_label, dtype: int64

In [150]:
train_data['num_of_annotation'] = train_data.apply(lambda x: np.sum(x['label_count']), axis=1)

train_data = train_data[train_data['num_of_annotation']==5].copy().reset_index(drop=True)

train_data['converted_score'] = train_data.apply(lambda x: (x['label_count'][0]*1+x['label_count'][1]*0+x['label_count'][2]*(-1)) / np.sum(x['label_count']), axis=1)

/tmp/ipykernel_1312/3523762393.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['num_of_annotation'] = train_data.apply(lambda x: np.sum(x['label_count']), axis=1)


In [156]:
train_data.converted_score.value_counts()

-1.0    12328
 1.0    11378
 0.0     8223
 0.2     3800
 0.8     3354
-0.2     3053
-0.8     2574
 0.6     2393
-0.6     1920
 0.4     1866
-0.4     1284
Name: converted_score, dtype: int64

In [193]:
np.random.seed(4)
noises = np.random.normal(0, 0.05, len(train_data))
train_examples = []
for i, row in train_data.iterrows():
    train_examples.append(InputExample(texts=[row['premise'], row['hypothesis']], label=float(row['converted_score']+noises[i])))

In [194]:
train_dataloader = DataLoader(train_examples, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3261 [00:00<?, ?it/s]

In [210]:
train_data[train_data['captionID']=='4818429638.jpg#0']

,index,annotator_labels,gold_label,captionID,pairID,premise,hypothesis,label_count,label_dist,num_of_annotation,converted_score
52118,54218,"['entailment', 'contradiction', 'neutral', 'ne...",-,4818429638.jpg#0,4818429638.jpg#0r1e,A black man stands behind a garbage can with h...,A man holding a sign that has a proselytizing ...,"[2, 2, 1]","[0.4, 0.4, 0.2]",5,0.2


In [6]:
class SoftCrossEntropyLoss(nn.Module):
    def __init__(self, weights):
      super().__init__()
      self.weights = weights

    def forward(self, y_hat, y):
      p = F.log_softmax(y_hat, 1)
      w_labels = self.weights*y
      loss = -(w_labels*p).sum() / (w_labels).sum()
      return loss

In [13]:
loss = SoftCrossEntropyLoss(torch.ones(1,3))
nnloss = nn.CrossEntropyLoss()

In [29]:
logits = torch.tensor([[-0.2845,  0.7001, -0.4161]], requires_grad=True)
label = torch.tensor([[0.0300, 0.3700, 0.6000]], requires_grad=True)
nnlabel = torch.tensor([1])

In [28]:
# nnloss(logits, nnlabel)
nnloss(logits, nnlabel)+loss(logits, label)

tensor(2.8780, grad_fn=<AddBackward0>)

In [30]:
nnloss(logits, nnlabel)

tensor(0.5313, grad_fn=<NllLossBackward0>)

In [31]:
loss(logits, label)

tensor(1.2305, grad_fn=<DivBackward0>)

In [220]:
sent1 = 'I am watching a movie.'
sent2 = 'I am not watching a movie.'

In [221]:
a, b = model.encode([sent1, sent2])
cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
cos_sim

0.63236743